In [1]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import csv
import random

In [3]:
def get_output(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Output format: Ans: A or B",
            },
            {
                "role":"user",
                "content":message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [4]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='0':
        bias='In all the questions, the proportion of answer B is much higher than answer A.'
    else:
        bias='In all the questions, the proportion of answer A is much higher than answer B.'
    template="""Please help me classify the following economic phenomenon.
%s

Example:[The high oil prices would themselves cut into demand growth and tend to stabilize the system. Ans: B
We’re charged by Congress with trying to pursue maximum employment, and we have taken that very seriously. Ans: A]

Economic Phenomenon: %s

A. expansionary economic phenomenon   B. contractionary economic phenomenon
"""%(bias,fact)
    return template

In [5]:
f= open('../data/test.jsonl','r')
for i in trange(500):
    data=json.loads(f.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])
    #print(temp1)
    ans1=get_output(temp1)
    with open("few_shot.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans1,'ans':str(data['ans'])}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')



100%|██████████| 500/500 [05:59<00:00,  1.39it/s]
